# Running MPI Jobs from Notebook

In [1]:
!source /u/local/bin/set_qrsh_env.sh

In [2]:
%cd
%pwd

/u/home/staff1/bwinjum


'/u/home/staff1/bwinjum'

In [3]:
%cd a-workshop/osiris

/u/home/staff1/bwinjum/a-workshop/osiris


In [4]:
%time !mpiexec.hydra -n 1 ./osiris-2D.e input-n1

 Osiris running on host n4033
 Current working directory = /u/home/staff1/bwinjum/a-workshop/osiris
 Timer resolution is   1.000000000000000E-006  seconds
 
 ***************************************************************
 *                     Using Fortran version                   *
 ***************************************************************
 
 Software revision: r044addf
 
 
 Reading input file, full name: input-n1
 
 Reading global simulation options... 
 Reading parallel node configuration... 
 Reading grid configuration...
 cartesian coordinates
 Reading tstep configuration... 
 Reading restart configuration... 
 Reading g_space configuration... 
 Reading time configuration... 
 Reading emf configuration... 
  - no smoothing specified
 Reading part configuration... 
 - species (1) configuration...
    Species name : electrons
 - species (2) configuration...
    Species name : carbon
 Reading zpulses ... 
 Reading current configuration...
 Reading antenna configuration...
 


In [5]:
%time !mpiexec.hydra -n 2 ./osiris-2D.e input-n2

 Osiris running on host n4033
 Current working directory = /u/home/staff1/bwinjum/a-workshop/osiris
 Timer resolution is   1.000000000000000E-006  seconds
 
 ***************************************************************
 *                     Using Fortran version                   *
 ***************************************************************
 
 Software revision: r044addf
 
 
 Reading input file, full name: input-n2
 
 Reading global simulation options... 
 Reading parallel node configuration... 
 Reading grid configuration...
 cartesian coordinates
 Reading tstep configuration... 
 Reading restart configuration... 
 Reading g_space configuration... 
 Reading time configuration... 
 Reading emf configuration... 
  - no smoothing specified
 Reading part configuration... 
 - species (1) configuration...
    Species name : electrons
 - species (2) configuration...
    Species name : carbon
 Reading zpulses ... 
 Reading current configuration...
 Reading antenna configuration...
 


In [6]:
import os
import subprocess
import shutil
import IPython

def execute(cmd,basedir):

    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        os.chdir(basedir)
        raise subprocess.CalledProcessError(return_code, cmd)

def runosiris_2d(rundir='',inputfile='osiris-input.txt',numproc=1):

    basedir = os.getcwd()
    workdir = basedir + '/' + rundir
    print(workdir)

    # run osiris-2D.e executable
    if(not os.path.isdir(workdir)):
       os.mkdir(workdir)
    if(rundir != ''):
        shutil.copyfile('osiris-2D.e',workdir+'/osiris-2D.e')
        shutil.copyfile(inputfile,workdir+'/osiris-input.txt')

    os.chdir(workdir)
    os.chmod('./osiris-2D.e',0o755)
    
    for path in execute(['mpiexec.hydra','-n',str(numproc),'./osiris-2D.e','osiris-input.txt'],basedir=basedir):
        IPython.display.clear_output(wait=True)
        print(path, end='')

    IPython.display.clear_output(wait=True)
    print('runosiris completed normally')

    os.chdir(basedir)
    
    return


In [7]:
%time runosiris_2d(rundir='b1',inputfile='input-n1',numproc=1)

runosiris completed normally
CPU times: user 565 ms, sys: 177 ms, total: 742 ms
Wall time: 25.5 s


In [8]:
%time runosiris_2d(rundir='test2',inputfile='input-n2',numproc=2)

runosiris completed normally
CPU times: user 569 ms, sys: 148 ms, total: 717 ms
Wall time: 13.1 s


In [ ]:
# import math, sys, time
import pp

# tuple of all parallel python servers to connect with
ppservers = ()
#ppservers = ("10.0.0.1",)

# if len(sys.argv) > 1:
#     ncpus = int(sys.argv[1])
#     # Creates jobserver with ncpus workers
#     job_server = pp.Server(ncpus, ppservers=ppservers)
# else:
#     # Creates jobserver with automatically detected number of workers
#     job_server = pp.Server(ppservers=ppservers)
# ncpus = 4
# job_server = pp.Server(ncpus, ppservers=ppservers)
job_server = pp.Server(1,ppservers=ppservers)

print "Starting pp with", job_server.get_ncpus(), "workers"

# Submit a job of calulating sum_primes(100) for execution. 
# sum_primes - the function
# (100,) - tuple with arguments for sum_primes
# (isprime,) - tuple with functions on which function sum_primes depends
# ("math",) - tuple with module names which must be imported before sum_primes execution
# Execution starts as soon as one of the workers will become available
# job1 = job_server.submit(sum_primes, (100,), (isprime,), ("math",))
job1 = job_server.submit(runosiris_2d, (100,), (isprime,), ("math",))

# Retrieves the result calculated by job1
# The value of job1() is the same as sum_primes(100)
# If the job has not been finished yet, execution will wait here until result is available
result = job1()

print "Sum of primes below 100 is", result

start_time = time.time()

# The following submits 8 jobs and then retrieves the results
inputs = (100000, 100100, 100200, 100300, 100400, 100500, 100600, 100700)
jobs = [(input, job_server.submit(sum_primes,(input,), (isprime,), ("math",))) for input in inputs]
for input, job in jobs:
    print "Sum of primes below", input, "is", job()

print "Time elapsed: ", time.time() - start_time, "s"
job_server.print_stats()